---
**Chapter 5: Introduction to Computer Theory by Daniel I. A. Cohen**

In [ ]:
#!pip install SNOBOL4python==0.4.5

In [ ]:
import re
from graphviz import Digraph
from itertools import product
from pprint import pformat, pprint
from IPython.display import display, Latex, Markdown
import sys
from pprint import pprint, pformat
## Thirty one (31) flavors of patterns to choose from ...
from SNOBOL4python import ε, σ, π, λ, Λ, ζ, θ, Θ, φ, Φ, α, ω
from SNOBOL4python import ABORT, ANY, ARB, ARBNO, BAL, BREAK, BREAKX, FAIL
from SNOBOL4python import FENCE, LEN, MARB, MARBNO, NOTANY, POS, REM, RPOS
from SNOBOL4python import RTAB, SPAN, SUCCESS, TAB
# Miscellaneous
from SNOBOL4python import GLOBALS, TRACE, PATTERN, STRING
from SNOBOL4python import ALPHABET, DIGITS, UCASE, LCASE, NULL
from SNOBOL4python import nPush, nInc, nPop, Shift, Reduce, Pop
# Instantiate the global variable space
GLOBALS(globals())
FA = dict()

In [ ]:
def display_dfa(fa):
    dot = Digraph()
    dot.attr(rankdir='LR')
    dot.attr(bgcolor='black')
    dot.attr('node', style='filled', fillcolor='black', fontcolor='white', color='white')
    dot.attr('edge', color='white', fontcolor='white')
    Σ = fa[0][1:]
    Δ = fa[1:]
    for i, δ in enumerate(Δ):
        q = δ[0]
        is_start = False
        is_finish = False
        if m := re.match(r"^\-(.*)$", q): is_start = True;  q = m.groups(1)[0]
        if m := re.match(r"^\+(.*)$", q): is_finish = True; q = m.groups(1)[0]
        if is_finish: dot.node(str(i+1), shape='doublecircle')
        else:         dot.node(str(i+1), shape='circle')
        if is_start:
            dot.node('', width='0', height='0', margin='0', shape='point', style='invis')
            dot.edge('', str(i+1), )
    for i, δ in enumerate(Δ):
        for j in range(1, 3):
            dot.edge(str(i+1), str(δ[j]), label=Σ[j-1])
    dot.render('dfa', format='png', cleanup=False)  # Saves as dfa.png
    display(dot)

In [ ]:
def compile_fa(fa_table):
    header = fa_table[0][1:]
    states = fa_table[1:]
    start_state = None
    accepting_states = set()
    transitions = {}
    for row in states:
        state_label = row[0]
        state_num = int(state_label.lstrip('+-'))
        if state_label.startswith('-'):
            start_state = state_num
        if state_label.startswith('+') or state_label.startswith('-+'):
            accepting_states.add(state_num)
        transitions[state_num] = {}
        for symbol, next_state in zip(header, row[1:]):
            transitions[state_num][symbol] = next_state
    if start_state is None:
        raise ValueError("No start state defined in FA table.")
    return start_state, header, transitions, accepting_states

In [ ]:
def run_fsm(input_str, start_state, header, transitions, accepting_states):
    current_state = start_state
    for ch in input_str:
        if ch not in header:
            raise ValueError(f"Invalid input symbol: {ch}")
        current_state = transitions[current_state][ch]
    accepted = current_state in accepting_states
    return accepted, current_state

In [ ]:
def show_samples(table):
    accepted = []
    rejected = []
    alphabet = table[0][1:]
    fsm = compile_fa(table)
    for length in range(0, 8):
        for combo in product(alphabet, repeat=length):
            word = ''.join(combo)
            is_accepted, _ = run_fsm(word, *fsm)
            if is_accepted:
                accepted.append(word)
            else: rejected.append(word)
    return accepted, rejected

In [ ]:
import textwrap
def display_problem(problem_name):
    problem = FA[problem_name]
    display(Latex(problem[0]))
    display_dfa(problem[2])
    accepted, rejected = show_samples(problem[2])
    print("Accepted:")
    print(textwrap.fill(" ".join(accepted), width=80))
    print()
    print("Rejected:")
    print(textwrap.fill(" ".join(rejected), width=80))

---
**Problem 1:** Write out the transition tables for the FAs on pp. 56, 58 (both), 63, 64, and 69 that were defined by pictures.

**Answer 1:** *See below*

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_56'] = (\
  "$(a+b)^{*}b(a+b)^{*}$"
, ARBNO(σ('a') | σ('b')) + σ('b') + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3]
  , [  '2',  1 ,  3]
  , [ '+3',  3 ,  3]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_56')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_58_1'] = (\
  "$(a+b)(a+b)^{*}$"
, SPAN('ab')
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [ '+2',  2 ,  2 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_58_1')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_58_2'] = (\
  "$(a+b)^{*}$"
, SPAN('ab') | ε()
, [ [  ' ', 'a', 'b']
  , ['-+1',  1 ,  1 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_58_2')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_63'] = (\
  "$(a+b)^{*}(aa+bb)(a+b)^{*}$"
, ARBNO(σ('a') | σ('b')) + (σ('aa') | σ('bb')) + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  4 ,  3 ]
  , [  '3',  2 ,  4 ]
  , [ '+4',  4 ,  4 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_63')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_64'] = (\
  "$(a+b)(a+b)b(a+b)*$"
, (σ('aaa') | σ('bbb')) + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  5 ]
  , [  '4',  4 ,  4 ]
  , [ '+5',  5 ,  5 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_64')

In [ ]:
# pg_69
na = nb = 0
def init(): na = nb = 0; return True
def acnt(): global na; na += 1; return True
def bcnt(): global nb; nb += 1; return True
FA['EVEN_EVEN'] = (\
  "$(aa+bb+(ab+ba)(aa+bb)^{*}(ab+ba))^{*}$"
, POS(0) + Λ("init()")
+ ARBNO(
    σ('a') + λ("acnt()")
  | σ('b') + λ("bcnt()")
  )
+ RPOS(0) + λ("a % 2 == 0 and b % 2 == 0")
, [ [  ' ', 'a', 'b']
  , ['-+1',  3 ,  2 ]
  , [  '2',  4 ,  1 ]
  , [  '3',  1 ,  4 ]
  , [  '4',  2 ,  3 ]
  ])
#-------------------------------------------------------------------------------
display_problem('EVEN_EVEN')

---
**Problem 2:** Build an FA that accepts only the language of all words with b as the second letter. Show both the picture and the transition table for this machine and find a regular expression for the language.

**Answer 2:**

RE = $(a + b)b(a+b)^{*}$

In [ ]:
#-------------------------------------------------------------------------------
FA['prob5_2'] = (\
  "$(a+b)b(a+b)^{*}$"
, ANY("ab") + σ('b') + ARBNO(σ('a') | σ('b'))
, [ [  ' ','a', 'b']
  , [ '-1', 2 ,  2 ]
  , [  '2', 3 ,  4 ]
  , [  '3', 3 ,  3 ]
  , [ '+4', 4 ,  4 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_2')

---
**Problem 3:** Build an FA that accepts only the words baa, ab, and abb and no other strings longer or shorter.

**Answer 3:**

RE = $(ab(Λ + b)+baa)$

In [ ]:
#-------------------------------------------------------------------------------
FA['prob5_3'] = (\
  "$(ab(Λ + b)+baa)$"
, σ('baa') | σ('ab') | σ('abb')
, [ [  ' ','a', 'b']
  , [ '-1', 2 ,  5 ]
  , [  '2', 8 ,  3 ]
  , [ '+3', 8 ,  4 ]
  , [ '+4', 8 ,  8 ]
  , [  '5', 6 ,  8 ]
  , [  '6', 7 ,  8 ]
  , [ '+7', 8 ,  8 ]
  , [  '8', 8 ,  8 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_3')

---
**Problem 4:**
* (i) Build an FA with the three states that accepts all strings.
* (ii) Show that given any FA with three states and three *+*'s, it accepts all input strings.
* (iii) If an FA has three states and only one *+*, must it reject some inputs?

**Answer 4:**
- *See below*
- follows from the pigeon-hole principle
- No.

In [ ]:
FA['prob5_4'] = (\
  "$(a+b)(a+b)(a+b)^{*}$"
, ARBNO(σ('a') | σ('b'))
, [ [ ' ' , 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [ '+2',  3 ,  3 ]
  , [ '+3',  3 ,  3 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_4')

---
**Problem 5:**
* (i) Build an FA that accepts only those words that have more that four letters.
* (ii) Build an FA that accepts only those words that have fewer than four letters.
* (iii) Build an FA that accepts only those words with exactly four letters.

**Answer 5:**
- RE: $\quad (a+b)^{5}(a+b)^{*}$
- RE: $\quad Λ+a+b+(Λ+a+b+(Λ+a+b)), \quad (Λ+a+b)^3$
- RE: $\quad (a+b)(a+b)(a+b)(a+b), \quad (a+b)^4$

In [ ]:
FA['prob5_5_1'] = (\
  "$(a+b)(a+b)(a+b)(a+b)(a+b)(a+b)^{*}$"
, SPAN("ab") @ "tx" + Λ("len(tx) > 4")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [  '5',  6 ,  6 ]
  , [ '+6',  6 ,  6 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_5_1')

In [ ]:
FA['prob5_5_2'] = (\
  "$(Λ+a+b)(Λ+a+b)(Λ+a+b)$"
, SPAN("ab") @ "tx" + Λ("len(tx) < 4")
, [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [ '+2',  3 ,  3 ]
  , [ '+3',  4 ,  4 ]
  , [ '+4',  5 ,  5 ]
  , [  '5',  5 ,  5 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_5_2')

In [ ]:
FA['prob5_5_3'] = (\
  "$(a+b)(a+b)(a+b)(a+b)$"
, SPAN("ab") @ "tx" + Λ("len(tx) == 4")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [ '+5',  6 ,  6 ]
  , [  '6',  6 ,  6 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_5_3')

---
**Problem 6:** Build an FA that accepts only those words that do *not* end with *ba*.

**Answer 6:**
RE: $Λ+a+(a+b)^{*}(b+aa)$

In [ ]:
FA['prob5_6'] = (\
  "$Λ+a+(a+b)^{*}(b+aa)$"
, ARBNO(σ('a') + σ('b')) + σ('ba') + RPOS(0) + FAIL()
, [ [  ' ' , 'a', 'b']
  , ['-+1' ,  1 ,  2 ]
  , [ '+2' ,  3 ,  2 ]
  , [  '3' ,  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_6')

---
**Problem 7:** Build an FA that accepts only those words that begin or end with a double letter.

**Answer 7:** RE: $\quad (aa+bb)(a+b)^{*}+(a+b)^{*}(aa+bb)$

In [ ]:
FA['prob5_7'] = (\
  "$(aa+bb)(a+b)^{*}+(a+b)^{*}(aa+bb)$"
, ( POS(0) + (σ('aa') | σ('bb')) + ARB()
  | (σ('aa') | σ('bb')) + ARB() + RPOS(0)
  )
, [ [   ' ', 'a', 'b']
  , [  '-1',  2 ,  4 ]
  , [   '2',  3 ,  6 ]
  , [  '+3',  3 ,  3 ]
  , [   '4',  7 ,  5 ]
  , [  '+5',  5 ,  5 ]
  , [   '6',  7 ,  8 ]
  , [   '7',  9 ,  6 ]
  , [  '+8',  7 ,  8 ]
  , [  '+9',  9 ,  7 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_7')

---
**Problem 8:** Build an FA that accepts only those words that have an even number of substrings *ab*.

**Answer 8: ???**
- RE: $\quad (abab)^{*}$
- RE: $\quad (aa^{*}bb^{*}aa^{*}bb^{*})^{*}$

In [ ]:
FA['prob5_8_1'] = (\
  "$(abab)^{*}$"
, ARBNO(σ('abab'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  6 ]
  , [  '2',  6 ,  3 ]
  , [  '3',  4 ,  6 ]
  , [  '4',  6 ,  5 ]
  , [ '+5',  2 ,  6 ]
  , [  '6',  6 ,  6 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_8_1')

In [ ]:
FA['prob5_8_2'] = (\
  "$(aa^{*}bb^{*}aa^{*}bb^{*})^{*}$"
, ARBNO(
    σ('a') + ARBNO(σ('a')) + σ('b') + ARBNO(σ('b'))
  + σ('a') + ARBNO(σ('a')) + σ('b') + ARBNO(σ('b'))
  )
, [ [   ' ', 'a', 'b']
  , [ '-+1',  2 ,  1 ]
  , [   '2',  2 ,  3 ]
  , [   '3',  4 ,  3 ]
  , [   '4',  4 ,  1 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_8_2')

---
**Problem 9:**
* (i) Recall from Chapter 4 the language of all words over the alphabet {*a b*} that have both the letter *a* and the letter *b* in them, but not necessarily in that order. Build an FA that accepts this language.
* (ii) Build an FA that accepts the language of all words with only *a*'s or only *b*'s in them. Give a regular expression for this language.

**Answer 9:**
- (i) RE: $(a+b)^{*}(ab+ba)(a+b)^{*}$ or $\quad (aa^{*}b+bb^{*}a)(a+b)^{*}$
- (ii) RE: $aa^{*}+bb^{*}$


In [ ]:
FA['prob5_9_1'] = (\
  "$(a+b)^{*}(ab+ba)(a+b)^{*}$"
, ARB() + σ('a') + ARB() + σ('b') + ARB()
| ARB() + σ('b') + ARB() + σ('a') + ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  2 ,  4 ]
  , [  '3',  4 ,  3 ]
  , [ '+4',  4 ,  4 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_9_1')

In [ ]:
FA['prob5_9_2'] = (\
  "$aa^{*}+bb^{*}$"
, SPAN("a") | SPAN("b")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [ '+2',  2 ,  4 ]
  , [ '+3',  4 ,  3 ]
  , [  '4',  4 ,  4 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_9_2')

---
**Problem 10:** Consider all the possible FAs over the alphabet { a, b } that have exactly two states. An FA must have a designated start state, but there are four possible ways to place the *+*'s:

Type 1: ['-',  '']
Type 2: ['-+', '']
Type 3: ['-',  '+']
Type 4: ['-+', '+']

Each FA needs four edges (two from each state), each of which can lead to either of the states. There are $2^{4}=16$ ways to arrange the labeled edges for each of the four types of FAs. Therefore, in total there are 64 different FAs of two states. However, they do not represent 64 non-equivalent FAs because they are not all associated with different languages. All type 1 FAs do not accept any words at all, whereas all FAs of type 4 accepts all strings of *a*'s and *b*'s.

* (i) Draw the remaining FAs of type 2.
* (ii) Draw the remaining FAs of type 3.
* (iii) Recalculate the total number of two-state machines using the transition table definition.

**Answer 10:**

In [ ]:
import itertools
FA_template = (\
  ""
, ARB()
, [ [ ' ', 'a', 'b']
  , [ '1',  0 ,  0 ]
  , [ '2',  0 ,  0 ]])
#-------------------------------------------------------------------------------
for t in range(1,3):
    for i, combo in enumerate(itertools.product([1, 2], repeat=4)):
        name = f"prob_10_{i}"
        if t == 1:
            FA_template[2][1][0] = '-+1'
            FA_template[2][2][0] = '2'
        if t == 2:
            FA_template[2][1][0] = '-1'
            FA_template[2][2][0] = '+2'
        FA_template[2][1][1] = combo[0]
        FA_template[2][1][2] = combo[1]
        FA_template[2][2][1] = combo[2]
        FA_template[2][2][2] = combo[3]
        FA[name] = FA_template
        display_problem(name)

---
**Problem 11:** Show that there are exactly 5832 different finite automata with three states *x* *y*, *z* over the alphabet {*a b*}, where *x* is always the start state.

**Answer 11:**

---
**Problem 12:**

**Answer 12:**

---
**Problem 13:**

**Answer 13:**

---
**Problem 14:**

**Answer 14:**

---
**Problem 15:** Build a machine that accepts all strings that have an even length that is not divisible by 6.

**Answer 15:**

In [ ]:
FA['prob5_15'] = (\
  "?"
, SPAN("ab") @ "tx" + Λ("len(tx) % 2 == 0 and len(tx) % 6 != 0")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [ '+3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [ '+5',  6 ,  6 ]
  , [  '6',  7 ,  7 ]
  , [  '7',  2 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_15')

---
**Problem 16:** Build an FA such that when the labels *a* and *b* are swapped the new machine is different from the old one but equivalent (the language defined by these machines is the same).

**Answer 16:**

---
**Problem 17:**

**Answer 17:**
- $(aa+ab+ba+bb)a$
- $(a+b)(aa+ab+ba+bb)a$
- $(aa+ba)(aa+ba)^{*}$

---
**Problem 18:**

**Answer 18:**

---
**Problem 19:**

**Answer 19:**

---
**Problem 20:**

**Answer 20:**